In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo

## Mars News

In [2]:
# Execute Scrape
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Get LATEST driver version for 90.0.4430
Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Paris\.wdm\drivers\chromedriver\win32\90.0.4430.24]


In [3]:
# Open browser to NASA Mars News Site
browser.visit('https://mars.nasa.gov/news/')

In [4]:
html = browser.html
soup = bs(html, 'html.parser')

# parse containing class 
results = soup.find_all(class_='slide')[0]

# get the latest tittle and body
news_title = results.find(class_='content_title').text
news_p = results.find(class_='article_teaser_body').text
print(news_title)
print(news_p)

NASA's Ingenuity Helicopter to Begin New Demonstration Phase
The Red Planet rotorcraft will shift focus from proving flight is possible on Mars to demonstrating flight operations that future aerial craft could utilize.


## Mars Featured Image

In [5]:
# Image Scrape
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\Paris\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [6]:
# Connect to JPL 
browser.visit('https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html')

In [7]:
# Click through to find full image
featured_image = browser.links.find_by_partial_text('FULL IMAGE').click()

In [9]:
html = browser.html
image_soup = bs(html, "html.parser")

In [8]:
image_loc = image_soup.find_all("div", class_="fancybox-inner")
image_loc

[<div class="fancybox-inner" style="overflow: visible; width: 640px; height: 350px;"><img alt="" class="fancybox-image" src="image/featured/mars1.jpg"/></div>]

In [10]:
featured_img_url = image_soup.find_all('img')[1]["src"]
featured_img_url

'image/featured/mars3.jpg'

In [11]:
# Print based URL + featured_img_url
final_image_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space'+featured_img_url
final_image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Spaceimage/featured/mars3.jpg'

## Mars Facts

In [12]:
# Open browser to NASA Mars News Site
mars_facts = pd.read_html('https://space-facts.com/mars/')
# get table with desired values
mars_facts = mars_facts[0]

In [13]:
mars_facts.columns=['properties', 'measurements']
mars_facts

,properties,measurements
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


## Mars Hemispheres

In [14]:
# Visit the USGS Astrogeology site to obtain high resolution images for each of Mar's hemispheres
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\Paris\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [15]:
# Parse HTML with BeautifulSoup
soup = bs(browser.html, 'html.parser')

# Create List to hold all the images & URLs collected
hems_image_url = []

#locate element that holds results for all hemis
results = soup.find_all('div', class_='item')

for result in results:
    # get tittle for hemis
    title_hemis = result.find('h3').text
    
    # = URL for the larger img page
    first_url = 'https://astrogeology.usgs.gov' + result.find('a', class_='itemLink')['href']
    
     
    #connect to first URL and parse
    browser.visit(first_url)
    soup = bs(browser.html, 'html.parser')
    
    # Open full image and get URL or src
    final_url = 'https://astrogeology.usgs.gov' + soup.find('img', class_='wide-image')['src']
    
    # Append list
    hems_image_url.append({'title': title_hemis, 'img_url':final_url})
    
    ##Go back
    browser.back()
    
hems_image_url

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]